# Gráficos CEPAL

In [1]:
import xml.etree.ElementTree as ET

tree_tamano_medio_hogares = ET.parse('Tamano_medio_hogares.xml')
root = tree_tamano_medio_hogares.getroot()

In [2]:
dims = ['País', 'Área geográfica', 'Quintil', 'Años']

identifiers  = {k : {} for k in dims}

for elem in root[2].iter():
    d = elem.attrib
    try:
        if d['name'] in dims:
            key = d['name']
            continue
        else:
            identifiers[key][d['id']] = d['name']
    except KeyError:
        pass
    

In [3]:
dict_tamano_medio_hogares = {}

dim_id = {'dim_208' : 'País',
         'dim_326' : 'Área geográfica',
         'dim_378' : 'Quintil',
         'dim_29117' : 'Años',
         'id_fuente' : 'id_fuente',
         'ids_notas' : 'ids_notas',
         'iso3' : 'iso3',
         'valor' : 'valor'}

for i, elem in enumerate(root[1].iter()):
    if i > 0:
        if i == 1:
            for key, item in elem.attrib.items():
                k = dim_id[key]
                try: 
                    dict_tamano_medio_hogares[k] = [identifiers[k][item]]
                except KeyError:
                    dict_tamano_medio_hogares[k] = [item]
        else:
            for key, item in elem.attrib.items():
                k = dim_id[key]
                try:
                    dict_tamano_medio_hogares[k].append(identifiers[k][item])
                except KeyError:
                    dict_tamano_medio_hogares[k].append(item)

In [4]:
import pandas as pd

data_frame = pd.DataFrame(dict_tamano_medio_hogares)

data_frame.head(20)

,País,Área geográfica,Quintil,Años,id_fuente,ids_notas,iso3,valor
0,América Latina (promedio simple),Nacional,Quintil 1,2001,26,"8620,7765",,5.2
1,América Latina (promedio simple),Nacional,Quintil 2,2001,26,"8620,7765",,4.8
2,América Latina (promedio simple),Nacional,Quintil 3,2001,26,"8620,7765",,4.3
3,América Latina (promedio simple),Nacional,Quintil 4,2001,26,"8620,7765",,3.8
4,América Latina (promedio simple),Nacional,Quintil 5,2001,26,"8620,7765",,3.2
5,América Latina (promedio simple),Nacional,Total quintiles,2001,26,"8620,7765",,4.3
6,América Latina (promedio simple),Rural,Quintil 1,2001,26,"8620,10395",,5.7
7,América Latina (promedio simple),Rural,Quintil 2,2001,26,"8620,10395",,5.2
8,América Latina (promedio simple),Rural,Quintil 3,2001,26,"8620,10395",,4.8
9,América Latina (promedio simple),Rural,Quintil 4,2001,26,"8620,10395",,4.2


In [5]:
data_frame[['Años','valor']] = data_frame[['Años','valor']].astype(dtype='float64')

filtered_year = data_frame[(data_frame['Años']==2002) |
                           (data_frame['Años']==2010) |
                           (data_frame['Años']==2019) ] 

filtered_year.to_csv('Databases/tamano_hogar.csv', index=False)
#gr_pais_v = data_frame.groupby(['País','Quintil']).mean()

#gr_pais_v.head(114)